# Architecture Decision Record (ADR) Generation

## What is a ADR?

ADRs are a best practice for documenting why an architecture is chosen. The cloud is an evolving product. The architectural choices made today are based on the features available to us. Features could be different tomorrow, so documenting the decision process allows later contributors to understand why systems are architected a certain way.

**Relevant links:** (alphabetical order)
 * https://adr.github.io/ 
 * https://cloud.google.com/architecture/architecture-decision-records
 * https://docs.aws.amazon.com/prescriptive-guidance/latest/architectural-decision-records
 * https://learn.microsoft.com/en-us/azure/architecture/guide/technology-choices (same concept)

## How can Lexical.cloud be utilized for architectural decisions?

Lexical.cloud is a data source about multi-cloud! By utilizing the same data from the analysis of a release, we can consider what tools are appropriate with the process of elimination. Just run `transform-lc.pynb` to generate the data required for following along.

**Disclaimer:** Decisions can only be as good as the data used to make them, so keep a lookout for data issues.

### Load the data

In [1]:
import pandas as pd

products_df = pd.read_json("data/products.json")
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277 entries, 0 to 276
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        277 non-null    object
 1   providers   277 non-null    object
 2   services    277 non-null    object
 3   domains     277 non-null    object
 4   categories  277 non-null    object
 5   features    66 non-null     object
 6   links       277 non-null    object
 7   type        277 non-null    object
 8   labels      3 non-null      object
dtypes: object(9)
memory usage: 19.6+ KB


### What type of service is being selected? **(Database)**

We'll consider all known clouds, but we could filter from the start.

In [2]:
#provider_type = 'aws'
service_type = 'database'

service_products_df = (
    products_df
        .query("type == 'product'")
        .explode('providers')
        #.query("providers == @provider_type")
        .explode('services')
        .query("services == @service_type")
)

service_products_df.head(5)

,name,providers,services,domains,categories,features,links,type,labels
51,Amazon Athena,aws,database,"[big data, data management, serverless]","[sql compliant, unstructured data]",NaN,{'self': '/products/database/aws/athena'},product,NaN
52,Amazon Aurora,aws,database,"[data management, managed service, serverless]","[relational db, sql compliant]","[mysql, postgresql]",{'self': '/products/database/aws/aurora'},product,NaN
53,Amazon DynamoDB,aws,database,"[data management, serverless]","[document db, key-value store, nosql db]",NaN,{'self': '/products/database/aws/dynamodb'},product,NaN
54,Amazon ElastiCache,aws,database,[managed service],[caching],NaN,{'self': '/products/database/aws/elasticache'},product,NaN
55,Amazon Relational Database Service,aws,database,"[data management, managed service]","[relational db, sql compliant]","[mysql, postgresql, mariadb, sql server, oracle]",{'self': '/products/database/aws/rds'},product,NaN


### What attributes do we have to compare?

Let's turn the entire taxonomy into our decision criteria.

In [10]:
# combine domains, categories, features and labels into one column for pivot
service_products_df['taxonomy'] = ( 
    service_products_df.domains +
    service_products_df.categories + 
    # the rest are known to be optional
    service_products_df.features.fillna("").apply(list) + 
    service_products_df.labels.fillna("").apply(list)
).map(set).map(list)

pivot_column = 'taxonomy'
pivot_products_df = (
    service_products_df
        .filter(items=['name',pivot_column])
        .explode(pivot_column)
        .groupby(['name',pivot_column]).size().reset_index(name='count')
        .pivot('name',pivot_column,'count').fillna(0, downcast='infer')
)

pivot_products_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22 entries, AWS Database Migration Service to Google Cloud Spanner
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   big data            22 non-null     int64
 1   caching             22 non-null     int64
 2   data management     22 non-null     int64
 3   data warehousing    22 non-null     int64
 4   database migration  22 non-null     int64
 5   document db         22 non-null     int64
 6   edge computing      22 non-null     int64
 7   key-value store     22 non-null     int64
 8   managed service     22 non-null     int64
 9   mariadb             22 non-null     int64
 10  mysql               22 non-null     int64
 11  nosql db            22 non-null     int64
 12  oracle              22 non-null     int64
 13  postgresql          22 non-null     int64
 14  relational db       22 non-null     int64
 15  serverless          22 non-null     int64
 16  sql 

### What attribute is most important for this scenario? **(SQL Compliant)**

There could be multiple, but we'll just start with one. The reason should be documented.

In [4]:
decision_requirement = 'sql compliant'
decision_matrix_df = (
    pivot_products_df
        .loc[pivot_products_df[decision_requirement] == 1]
)

decision_matrix_df

taxonomy,big data,caching,data management,data warehousing,database migration,document db,edge computing,key-value store,managed service,mariadb,mysql,nosql db,oracle,postgresql,relational db,serverless,sql compliant,sql server,unstructured data
name,,,,,,,,,,,,,,,,,,,
Amazon Athena,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1
Amazon Aurora,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1,1,1,0,0
Amazon Relational Database Service,0,0,1,0,0,0,0,0,1,1,1,0,1,1,1,0,1,1,0
Azure Database for MySQL,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0
Azure Database for PostgreSQL,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0
Azure SQL Database,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0
Azure SQL Edge,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0
Azure SQL Managed Instance,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0
Google Cloud BigQuery,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1


## Begin the Decision Record

All decisions made to this point should be documented here.

### What tradeoffs exist for our scenario? **(SQL Databases)**

Let's discard columns with no differences and start considering the best SQL compliant database for our purpose.

In [5]:
adr = (
    decision_matrix_df
        # drop columns with all 0
        .loc[:, (decision_matrix_df != 0).any(axis=0)]
        # drop columns with all 1
        .loc[:, (decision_matrix_df != 1).any(axis=0)]
        # use Y or N not 1 or 0
        .replace({0: 'N', 1: 'Y' })
)

adr

taxonomy,big data,data warehousing,edge computing,managed service,mariadb,mysql,oracle,postgresql,relational db,serverless,sql server,unstructured data
name,,,,,,,,,,,,
Amazon Athena,Y,N,N,N,N,N,N,N,N,Y,N,Y
Amazon Aurora,N,N,N,Y,N,Y,N,Y,Y,Y,N,N
Amazon Relational Database Service,N,N,N,Y,Y,Y,Y,Y,Y,N,Y,N
Azure Database for MySQL,N,N,N,Y,N,Y,N,N,Y,N,N,N
Azure Database for PostgreSQL,N,N,N,Y,N,N,N,Y,Y,N,N,N
Azure SQL Database,N,N,N,Y,N,N,N,N,Y,Y,N,N
Azure SQL Edge,N,N,Y,N,N,N,N,N,Y,N,N,N
Azure SQL Managed Instance,N,N,N,Y,N,N,N,N,Y,N,N,N
Google Cloud BigQuery,Y,Y,N,N,N,N,N,N,N,Y,N,Y


### What other requirements are there for this scenario? **(Serverless)**

We would document the reasoning before proceeding.

In [6]:
adr = adr.query("serverless == 'Y'")

adr

taxonomy,big data,data warehousing,edge computing,managed service,mariadb,mysql,oracle,postgresql,relational db,serverless,sql server,unstructured data
name,,,,,,,,,,,,
Amazon Athena,Y,N,N,N,N,N,N,N,N,Y,N,Y
Amazon Aurora,N,N,N,Y,N,Y,N,Y,Y,Y,N,N
Azure SQL Database,N,N,N,Y,N,N,N,N,Y,Y,N,N
Google Cloud BigQuery,Y,Y,N,N,N,N,N,N,N,Y,N,Y


### What other requirements? **(MySQL)**

We would document the reasoning before proceeding.

In [7]:
adr = adr.query("mysql == 'Y'")

adr

taxonomy,big data,data warehousing,edge computing,managed service,mariadb,mysql,oracle,postgresql,relational db,serverless,sql server,unstructured data
name,,,,,,,,,,,,
Amazon Aurora,N,N,N,Y,N,Y,N,Y,Y,Y,N,N


## Decision Reached!

Let's review all the attributes of what was selected.

In [8]:
decision = (
    decision_matrix_df
        .loc[adr.index[0]]
        .replace({0: 'N', 1: 'Y' })
        .to_frame()
)

decision

,Amazon Aurora
taxonomy,
big data,N
caching,N
data management,Y
data warehousing,N
database migration,N
document db,N
edge computing,N
key-value store,N
managed service,Y


### Satisfied? 

Document it by exporting and version controling information from this notebook! The available services could be different tomorrow. The system requirements could be different as well. Archiving the state of system requirements and the available tooling is a must.

# Stay tuned!

There is a plan to make this functionality more developer friendly. Coming soon to an IDE near you :)